In [ ]:
from demo import get_train_data

dataset=get_train_data()

In [ ]:
from transformers import BeitFeatureExtractor, BeitForImageClassification
feature_extractor = BeitFeatureExtractor.from_pretrained('microsoft/beit-base-patch16-224')
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224')

In [ ]:
import faiss

def embed(ex, idx=None, add_index=True):
    if add_index:
        images = ex["image"]
    else:
        images = ex
    inputs = feature_extractor(images=images, return_tensors="pt")
    outputs = model(**inputs,output_hidden_states= True)
    last_hidden=outputs.hidden_states[-1]
    pooler=model.base_model.pooler
    final_emb=pooler(last_hidden).detach().numpy()
    
    if add_index:
        ex['idx'] = idx
        index.add(final_emb)                  # add vectors to the index
    else: 
        dist,idx=index.search(final_emb,idx) 
        return dist,idx

In [ ]:
d= 768
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
dataset=dataset.map(embed,batched=True,batch_size=20,with_indices=True)
print(index.ntotal)

In [ ]:
from IPython.display import display

In [ ]:
query_image=dataset[3]
distances,res_ids=embed(query_image['image'],5,add_index=False)

In [ ]:
query_image["image"]

In [ ]:
for x in res_ids[0]:
    print(x)
    display(dataset[int(x)]["image"])


In [ ]:
print(index.ntotal)
